In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

2023-12-13 13:09:19.459240: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 13:09:19.459278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 13:09:19.460640: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 13:09:19.468690: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 13:09:20.527868: W tensorflow/compiler/tf2

In [2]:
tf.sysconfig.get_build_info() 

OrderedDict([('cpu_compiler', '/usr/lib/llvm-17/bin/clang'),
             ('cuda_compute_capabilities',
              ['sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']),
             ('cuda_version', '12.2'),
             ('cudnn_version', '8'),
             ('is_cuda_build', True),
             ('is_rocm_build', False),
             ('is_tensorrt_build', True)])

In [3]:
directory = "dataset"

In [4]:
train_data = tf.keras.utils.audio_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="categorical",
    batch_size=1,
    sampling_rate=None,
    output_sequence_length=8000,
    ragged=False,
    shuffle=True,
    validation_split=None,
    subset=None
)

Found 494 files belonging to 10 classes.


2023-12-13 13:09:34.196842: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-13 13:09:34.617061: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-13 13:09:34.617269: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [18]:
model = Sequential([
    Conv1D(32, kernel_size=80, strides=16, input_shape=(8000, 1), activation='relu'),
    # BatchNormalization(),
    MaxPooling1D(4),
    Conv1D(32, kernel_size=3, activation='relu'),
    # BatchNormalization(),
    MaxPooling1D(4),
    Conv1D(2 * 32, kernel_size=3, activation='relu'),
    # BatchNormalization(),
    MaxPooling1D(4),
    Conv1D(2 * 32, kernel_size=3, activation='relu'),
    # BatchNormalization(),
    MaxPooling1D(4),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax', input_shape=(None, 10))
])

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=256, activation='relu', input_shape=(8000, 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(128, kernel_size=256, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [19]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_14 (Conv1D)          (None, 496, 32)           2592      
                                                                 
 max_pooling1d_14 (MaxPooli  (None, 124, 32)           0         
 ng1D)                                                           
                                                                 
 conv1d_15 (Conv1D)          (None, 122, 32)           3104      
                                                                 
 max_pooling1d_15 (MaxPooli  (None, 30, 32)            0         
 ng1D)                                                           
                                                                 
 conv1d_16 (Conv1D)          (None, 28, 64)            6208      
                                                                 
 max_pooling1d_16 (MaxPooli  (None, 7, 64)            

In [20]:
model.compile(
    optimizer=Adam(), 
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [21]:
model.fit(train_data, epochs=40, batch_size=4)

Epoch 1/40


494/494 [==============================] - 5s 5ms/step - loss: 2.1248 - accuracy: 0.1862
Epoch 2/40
494/494 [==============================] - 2s 5ms/step - loss: 1.9057 - accuracy: 0.2591
Epoch 3/40
494/494 [==============================] - 2s 5ms/step - loss: 1.7377 - accuracy: 0.3381
Epoch 4/40
494/494 [==============================] - 2s 5ms/step - loss: 1.6079 - accuracy: 0.3623
Epoch 5/40
494/494 [==============================] - 2s 5ms/step - loss: 1.4565 - accuracy: 0.4211
Epoch 6/40
494/494 [==============================] - 2s 5ms/step - loss: 1.3725 - accuracy: 0.4798
Epoch 7/40
494/494 [==============================] - 2s 5ms/step - loss: 1.2855 - accuracy: 0.5142
Epoch 8/40
494/494 [==============================] - 2s 5ms/step - loss: 1.1155 - accuracy: 0.5709
Epoch 9/40
494/494 [==============================] - 2s 5ms/step - loss: 1.0176 - accuracy: 0.6356
Epoch 10/40
494/494 [==============================] - 2s 5ms/step - loss: 0.9365 - accuracy: 0.6397
Epoch 11/4